https://discourse.julialang.org/t/why-isnt-a-simple-map-type-stable/67028

https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-captured-1

In [1]:
function gen1(n)
    prevx = 0.0
    map(1:n) do _
        x = randn()
        r = prevx + x
        prevx = x
        r
    end
end

@code_warntype gen1(10)

Variables
  #self#::Core.Const(gen1)
  n::Int64
  #1::var"#1#2"
  prevx::Core.Box

Body::Vector{_A} where _A
1 ─      (prevx = Core.Box())
│        Core.setfield!(prevx, :contents, 0.0)
│        (#1 = %new(Main.:(var"#1#2"), prevx))
│   %4 = #1::var"#1#2"
│   %5 = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│   %6 = Main.map(%4, %5)::Vector{_A} where _A
└──      return %6


In [2]:
function gen1_rev1(n)
    prevx = fill(0.0)
    map(1:n) do _
        x = randn()
        r = prevx[] + x
        prevx[] = x
        r
    end
end

@code_warntype gen1_rev1(10)

Variables
  #self#::Core.Const(gen1_rev1)
  n::Int64
  #3::var"#3#4"{Array{Float64, 0}}
  prevx::Array{Float64, 0}

Body::Vector{Float64}
1 ─      (prevx = Main.fill(0.0))
│   %2 = Main.:(var"#3#4")::Core.Const(var"#3#4")
│   %3 = Core.typeof(prevx)::Core.Const(Array{Float64, 0})
│   %4 = Core.apply_type(%2, %3)::Core.Const(var"#3#4"{Array{Float64, 0}})
│        (#3 = %new(%4, prevx))
│   %6 = #3::var"#3#4"{Array{Float64, 0}}
│   %7 = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│   %8 = Main.map(%6, %7)::Vector{Float64}
└──      return %8


In [3]:
function gen1_rev2(n)
    prevx = Ref(0.0)
    map(1:n) do _
        x = randn()
        r = prevx[] + x
        prevx[] = x
        r
    end
end

@code_warntype gen1_rev2(10)

Variables
  #self#::Core.Const(gen1_rev2)
  n::Int64
  #5::var"#5#6"{Base.RefValue{Float64}}
  prevx::Base.RefValue{Float64}

Body::Vector{Float64}
1 ─      (prevx = Main.Ref(0.0))
│   %2 = Main.:(var"#5#6")::Core.Const(var"#5#6")
│   %3 = Core.typeof(prevx)::Core.Const(Base.RefValue{Float64})
│   %4 = Core.apply_type(%2, %3)::Core.Const(var"#5#6"{Base.RefValue{Float64}})
│        (#5 = %new(%4, prevx))
│   %6 = #5::var"#5#6"{Base.RefValue{Float64}}
│   %7 = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│   %8 = Main.map(%6, %7)::Vector{Float64}
└──      return %8


In [4]:
function gen1_rev3(n)
    prevx::Float64 = 0.0
    map(1:n) do _
        x = randn()
        r = prevx + x
        prevx = x
        r
    end
end

@code_warntype gen1_rev3(10)

Variables
  #self#::Core.Const(gen1_rev3)
  n::Int64
  #7::var"#7#8"
  prevx::Core.Box

Body::Vector{Float64}
1 ─      (prevx = Core.Box())
│   %2 = prevx::Core.Box
│   %3 = Base.convert(Main.Float64, 0.0)::Core.Const(0.0)
│   %4 = Core.typeassert(%3, Main.Float64)::Core.Const(0.0)
│        Core.setfield!(%2, :contents, %4)
│        (#7 = %new(Main.:(var"#7#8"), prevx))
│   %7 = #7::var"#7#8"
│   %8 = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│   %9 = Main.map(%7, %8)::Vector{Float64}
└──      return %9
